In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# import and modify data
npf_train = pd.read_csv("npf_train.csv")
npf_train = npf_train.drop("partlybad",axis=1)

npf_train = npf_train.set_index("date")
## Tell that "class4" is categorical variable. (R does this automatically.)
npf_train["class4"] = npf_train["class4"].astype("category")

## Here date column was converted to index and we do not need to get rid of it.
npf_train = npf_train.drop("id",axis=1)

## If you don't use dtype="object" array will cut strings...
class2 = np.array(["event"]*npf_train.shape[0],dtype="object")
class2[npf_train["class4"]=="nonevent"] = "nonevent"
npf_train["class2"] = class2
npf_train["class2"] = npf_train["class2"].astype("category")


# reformating X to numerics
for i in range(1,101): # 101 because we have 102 columns
    npf_train.iloc[:,i] = pd.to_numeric(list(npf_train.iloc[:,i]))

## REDUCING the set to get a training and a test set (360 ~ 80% of 458)
X = npf_train.iloc[:360, 1:-1]
y = npf_train.iloc[:360, -1]

# our test set:
X_test = npf_train.iloc[360:, 1:-1]
y_test = npf_train.iloc[360:, -1]

In [3]:
# train logistic regression model (a basic one for reference)
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression()
t_model_lr = model_lr.fit(X, y)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [4]:
# computing the score
score_lr = t_model_lr.score(X_test, y_test)
score_lr



0.8673469387755102

In [5]:
# retrain with higher max_iter & computing score
model_lr_higher_maxiter = LogisticRegression(max_iter= 10000)
t_model_lr_higher_maxiter = model_lr_higher_maxiter.fit(X,y)
t_model_lr_higher_maxiter_score = t_model_lr_higher_maxiter.score(X_test, y_test) 
t_model_lr_higher_maxiter_score

0.8469387755102041

In [6]:
# creating a function outputting the results of the model
def pred_proba_class(t_model, X_test_data, y_test_data): # t_model is the trained model
    df = pd.DataFrame()
    df["X"] = [X_test_data.iloc[i, :] for i in range(len(X_test_data))]
    df["y_true"] = np.array(y_test_data)
    df["y_pred_class"] = t_model.predict(X_test_data)
    proba_event= t_model.predict_proba(X_test_data)[:,0]
    df["y_pred_proba"] = proba_event
    df["correct?"] = [df.iloc[i,1] == df.iloc[i,2] for i in range(len(df))]
    
    return df

pred_proba_class(t_model_lr_higher_maxiter, X_test, y_test)

,X,y_true,y_pred_class,y_pred_proba,correct?
0,CO2168.mean 375.647561 CO2168.std 3.2...,event,event,0.925932,True
1,CO2168.mean 382.823235 CO2168.std 3.8...,event,event,0.996606,True
2,CO2168.mean 384.067500 CO2168.std 2.5...,event,event,0.990653,True
3,CO2168.mean 384.472708 CO2168.std 1.9...,event,event,0.999330,True
4,CO2168.mean 390.631489 CO2168.std 0.8...,nonevent,nonevent,0.028212,True
...,...,...,...,...,...
93,CO2168.mean 377.541538 CO2168.std 6.3...,nonevent,nonevent,0.280123,True
94,CO2168.mean 381.016623 CO2168.std 4.4...,nonevent,nonevent,0.000559,True
95,CO2168.mean 386.687895 CO2168.std 12.0...,nonevent,nonevent,0.005242,True
96,CO2168.mean 379.279128 CO2168.std 12.0...,nonevent,nonevent,0.006661,True


In [7]:
# modifying the logistic regression trying to improve the score:

# cross validation for evaluation 




###### predictor selection
# lbfgs and liblinear are not to be used since we want to be able to extend our model to class4 prediction

# Ridge Regression = penalty l2 --> default
model_lr_rr_newton = LogisticRegression(penalty = "l2", solver = 'newton-cg')
t_model_lr_rr_newton = model_lr_rr_newton.fit(X,y)
score_lr_rr_newton = t_model_lr_rr_newton.score(X_test, y_test)

model_lr_rr_sag = LogisticRegression(penalty = "l2", solver = 'sag')
t_model_lr_rr_sag = model_lr_rr_sag.fit(X,y)
score_lr_rr_sag = t_model_lr_rr_sag.score(X_test, y_test)

model_lr_rr_saga = LogisticRegression(penalty = "l2", solver = 'saga')
t_model_lr_rr_saga = model_lr_rr_saga.fit(X,y)
score_lr_rr_saga = t_model_lr_rr_saga.score(X_test, y_test)

# Lasso = penalty l1 
model_lr_lasso = LogisticRegression(penalty = "l1", solver = 'saga')# this solver bc it is the only option for multiclass & l1
t_model_lr_lasso = model_lr_lasso.fit(X,y)
score_lr_lasso = t_model_lr_lasso.score(X_test, y_test)

# none= penalty none
model_lr_none_newton = LogisticRegression(penalty = "none", solver = 'newton-cg')
t_model_lr_none_newton = model_lr_none_newton.fit(X,y)
score_lr_none_newton = t_model_lr_none_newton.score(X_test, y_test)

model_lr_none_sag = LogisticRegression(penalty = "none", solver = 'sag')
t_model_lr_none_sag = model_lr_none_sag.fit(X,y)
score_lr_none_sag = t_model_lr_none_sag.score(X_test, y_test)

model_lr_none_saga = LogisticRegression(penalty = "none", solver = 'saga')
t_model_lr_none_saga = model_lr_none_saga.fit(X,y)
score_lr_none_saga = t_model_lr_none_saga.score(X_test, y_test)

# Lasso + Ridge = ??? = penalty elasticnet
model_lr_elastic = LogisticRegression(penalty = "elasticnet", solver = 'saga', l1_ratio = 0.5) # this solver bc it's the only option, l1_ratio just in the middle for now
t_model_lr_elastic = model_lr_elastic.fit(X,y)
score_lr_elastic = t_model_lr_elastic.score(X_test, y_test)


# parameter tuning



# df for modifications, t_mode and score
mms = pd.DataFrame()
mms["modifications"] = ['RR newton', 'RR sag', 'RR saga', 'Lasso saga', 'none newton', 'none sag', 'none saga', 'Lasso+RR saga']
mms["t_models"] = [t_model_lr_rr_newton, t_model_lr_rr_sag, t_model_lr_rr_saga, t_model_lr_lasso, t_model_lr_none_newton, t_model_lr_none_sag, t_model_lr_none_saga, t_model_lr_elastic]
mms["simple scores"] = [score_lr_rr_newton, score_lr_rr_sag, score_lr_rr_saga, score_lr_lasso, score_lr_none_newton, score_lr_none_sag, score_lr_none_saga, score_lr_elastic]
mms

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means th

,modifications,t_models,simple scores
0,RR newton,LogisticRegression(solver='newton-cg'),0.846939
1,RR sag,LogisticRegression(solver='sag'),0.897959
2,RR saga,LogisticRegression(solver='saga'),0.846939
3,Lasso saga,"LogisticRegression(penalty='l1', solver='saga')",0.846939
4,none newton,"LogisticRegression(penalty='none', solver='new...",0.683673
5,none sag,"LogisticRegression(penalty='none', solver='sag')",0.887755
6,none saga,"LogisticRegression(penalty='none', solver='saga')",0.846939
7,Lasso+RR saga,"LogisticRegression(l1_ratio=0.5, penalty='elas...",0.846939


In [8]:
# same thing with maxiter = 1000 --> since there was a worning ...  to see if we get an improvement
# tried with max_iter = 1000 and = 10000 (gave warnings but terminated)
# --> results with 1000 ha better highest value thus here chosen:

# Ridge Regression = penalty l2 --> default
model_lr_rr_newton = LogisticRegression(penalty = "l2", solver = 'newton-cg', max_iter = 1000)
t_model_lr_rr_newton = model_lr_rr_newton.fit(X,y)
score_lr_rr_newton = t_model_lr_rr_newton.score(X_test, y_test)

model_lr_rr_sag = LogisticRegression(penalty = "l2", solver = 'sag',  max_iter = 1000)
t_model_lr_rr_sag = model_lr_rr_sag.fit(X,y)
score_lr_rr_sag = t_model_lr_rr_sag.score(X_test, y_test)

model_lr_rr_saga = LogisticRegression(penalty = "l2", solver = 'saga',  max_iter = 1000)
t_model_lr_rr_saga = model_lr_rr_saga.fit(X,y)
score_lr_rr_saga = t_model_lr_rr_saga.score(X_test, y_test)

# Lasso = penalty l1 
model_lr_lasso = LogisticRegression(penalty = "l1", solver = 'saga',  max_iter = 1000)# this solver bc it is the only option for multiclass & l1
t_model_lr_lasso = model_lr_lasso.fit(X,y)
score_lr_lasso = t_model_lr_lasso.score(X_test, y_test)

# none= penalty none
model_lr_none_newton = LogisticRegression(penalty = "none", solver = 'newton-cg',  max_iter = 1000)
t_model_lr_none_newton = model_lr_none_newton.fit(X,y)
score_lr_none_newton = t_model_lr_none_newton.score(X_test, y_test)

model_lr_none_sag = LogisticRegression(penalty = "none", solver = 'sag',  max_iter = 1000)
t_model_lr_none_sag = model_lr_none_sag.fit(X,y)
score_lr_none_sag = t_model_lr_none_sag.score(X_test, y_test)

model_lr_none_saga = LogisticRegression(penalty = "none", solver = 'saga',  max_iter = 1000)
t_model_lr_none_saga = model_lr_none_saga.fit(X,y)
score_lr_none_saga = t_model_lr_none_saga.score(X_test, y_test)

# Lasso + Ridge = ??? = penalty elasticnet
model_lr_elastic = LogisticRegression(penalty = "elasticnet", solver = 'saga', l1_ratio = 0.5,  max_iter = 1000) # this solver bc it's the only option, l1_ratio just in the middle for now
t_model_lr_elastic = model_lr_elastic.fit(X,y)
score_lr_elastic = t_model_lr_elastic.score(X_test, y_test)


# further parameter tuning possible



# df for modifications, t_mode and score
mmsMI = pd.DataFrame()
mmsMI["modifications"] = ['RR newton', 'RR sag', 'RR saga', 'Lasso saga', 'none newton', 'none sag', 'none saga', 'Lasso+RR saga']
mmsMI["t_models"] = [t_model_lr_rr_newton, t_model_lr_rr_sag, t_model_lr_rr_saga, t_model_lr_lasso, t_model_lr_none_newton, t_model_lr_none_sag, t_model_lr_none_saga, t_model_lr_elastic]
mmsMI["simple scores"] = [score_lr_rr_newton, score_lr_rr_sag, score_lr_rr_saga, score_lr_lasso, score_lr_none_newton, score_lr_none_sag, score_lr_none_saga, score_lr_elastic]
mmsMI

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/linesearch

,modifications,t_models,simple scores
0,RR newton,"LogisticRegression(max_iter=1000, solver='newt...",0.846939
1,RR sag,"LogisticRegression(max_iter=1000, solver='sag')",0.867347
2,RR saga,"LogisticRegression(max_iter=1000, solver='saga')",0.887755
3,Lasso saga,"LogisticRegression(max_iter=1000, penalty='l1'...",0.887755
4,none newton,"LogisticRegression(max_iter=1000, penalty='non...",0.673469
5,none sag,"LogisticRegression(max_iter=1000, penalty='non...",0.867347
6,none saga,"LogisticRegression(max_iter=1000, penalty='non...",0.887755
7,Lasso+RR saga,"LogisticRegression(l1_ratio=0.5, max_iter=1000...",0.887755


In [9]:
# training the upper models using cross validation
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from sklearn.model_selection import cross_val_score

cv10 = []
cv50 = []

for model in mms["t_models"]:
    cv = cross_val_score(model, X,y, cv = 10)
    cv10.append(cv)
    cv = cross_val_score(model, X,y, cv = 50)
    cv50.append(cv)
    
mms["cv score 10 fold"] = cv10
mms["cv score 50 fold"] = cv50

In [12]:
mms

,modifications,t_models,simple scores,cv score 10 fold,cv score 50 fold
0,RR newton,LogisticRegression(solver='newton-cg'),0.846939,"[0.7777777777777778, 0.8333333333333334, 0.694...","[0.625, 0.75, 0.875, 0.75, 1.0, 0.75, 0.75, 0...."
1,RR sag,LogisticRegression(solver='sag'),0.897959,"[0.8888888888888888, 0.9166666666666666, 0.722...","[1.0, 0.875, 0.875, 0.75, 1.0, 0.875, 0.75, 1...."
2,RR saga,LogisticRegression(solver='saga'),0.846939,"[0.8611111111111112, 0.9166666666666666, 0.722...","[1.0, 0.75, 0.875, 0.75, 0.875, 1.0, 0.75, 1.0..."
3,Lasso saga,"LogisticRegression(penalty='l1', solver='saga')",0.846939,"[0.8611111111111112, 0.9166666666666666, 0.722...","[1.0, 0.75, 0.875, 0.75, 0.875, 1.0, 0.75, 1.0..."
4,none newton,"LogisticRegression(penalty='none', solver='new...",0.683673,"[0.7777777777777778, 0.8888888888888888, 0.611...","[0.75, 0.75, 0.875, 0.75, 0.875, 0.875, 0.75, ..."
5,none sag,"LogisticRegression(penalty='none', solver='sag')",0.887755,"[0.8888888888888888, 0.9166666666666666, 0.722...","[1.0, 0.875, 0.875, 0.75, 1.0, 0.875, 0.75, 1...."
6,none saga,"LogisticRegression(penalty='none', solver='saga')",0.846939,"[0.8611111111111112, 0.9166666666666666, 0.722...","[1.0, 0.75, 0.875, 0.75, 0.875, 1.0, 0.75, 1.0..."
7,Lasso+RR saga,"LogisticRegression(l1_ratio=0.5, penalty='elas...",0.846939,"[0.8611111111111112, 0.9166666666666666, 0.722...","[1.0, 0.75, 0.875, 0.75, 0.875, 1.0, 0.75, 1.0..."


In [15]:
# mean cv scores:
cvmean_score = []

for i in range(len(mms)):
    cvmeans = sum(mms.iloc[i,3]) / len(mms.iloc[i,3])
    cvmean_score.append(cvmeans)
    
mms["cv mean score 10"] = cvmean_score


cvmean_score = []

for i in range(len(mms)):
    cvmeans = sum(mms.iloc[i,4]) / len(mms.iloc[i,4])
    cvmean_score.append(cvmeans)
    
mms["cv mean score 50"] = cvmean_score

mms

,modifications,t_models,simple scores,cv score 10 fold,cv score 50 fold,cv mean score 10,cv mean score 50
0,RR newton,LogisticRegression(solver='newton-cg'),0.846939,"[0.7777777777777778, 0.8333333333333334, 0.694...","[0.625, 0.75, 0.875, 0.75, 1.0, 0.75, 0.75, 0....",0.800000,0.796786
1,RR sag,LogisticRegression(solver='sag'),0.897959,"[0.8888888888888888, 0.9166666666666666, 0.722...","[1.0, 0.875, 0.875, 0.75, 1.0, 0.875, 0.75, 1....",0.813889,0.822857
2,RR saga,LogisticRegression(solver='saga'),0.846939,"[0.8611111111111112, 0.9166666666666666, 0.722...","[1.0, 0.75, 0.875, 0.75, 0.875, 1.0, 0.75, 1.0...",0.808333,0.808929
3,Lasso saga,"LogisticRegression(penalty='l1', solver='saga')",0.846939,"[0.8611111111111112, 0.9166666666666666, 0.722...","[1.0, 0.75, 0.875, 0.75, 0.875, 1.0, 0.75, 1.0...",0.808333,0.808929
4,none newton,"LogisticRegression(penalty='none', solver='new...",0.683673,"[0.7777777777777778, 0.8888888888888888, 0.611...","[0.75, 0.75, 0.875, 0.75, 0.875, 0.875, 0.75, ...",0.783333,0.765357
5,none sag,"LogisticRegression(penalty='none', solver='sag')",0.887755,"[0.8888888888888888, 0.9166666666666666, 0.722...","[1.0, 0.875, 0.875, 0.75, 1.0, 0.875, 0.75, 1....",0.813889,0.822857
6,none saga,"LogisticRegression(penalty='none', solver='saga')",0.846939,"[0.8611111111111112, 0.9166666666666666, 0.722...","[1.0, 0.75, 0.875, 0.75, 0.875, 1.0, 0.75, 1.0...",0.808333,0.808929
7,Lasso+RR saga,"LogisticRegression(l1_ratio=0.5, penalty='elas...",0.846939,"[0.8611111111111112, 0.9166666666666666, 0.722...","[1.0, 0.75, 0.875, 0.75, 0.875, 1.0, 0.75, 1.0...",0.808333,0.808929


In [ ]:
# --> none sag & RR sag with accuracy 81.3/82.2